In [112]:
# Bring in all of our dependencies
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import gym
import math



In [113]:
# We should define the environment here and detremine the sizing so we can generalize our simple neural network
env    = gym.make('CartPole-v0')   # Choose our environment
tmp    = env.action_space.sample()
print(type(tmp))
action = np.array([tmp])
N      = len(action)               # Use this to generalize the input size for the NN

[2016-11-23 20:24:22,213] Making new env: CartPole-v0


<class 'int'>


In [193]:
# Make a very simple neural network, reLU layers have been shown to train faster. We only have one hidden layer here, very simple
model = Sequential()
model.add(Dense(100, input_shape=((4,)), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(N))

In [194]:
# Build the model for training, we can tweak our loss function and optimizer for different environments
model.compile(loss='mean_squared_error', optimizer='sgd')

In [205]:
# Initialize replay memory
#repMem = np.empty((0,4))
gamma = 0.5
epsUpdate = 0.1
obs1 = env.reset()
action = env.action_space.sample()
obs2, reward, done, info = env.step(action)
repMem = np.array([obs1,action,reward,obs2])

#repMem = [1,2,3,4]
# Run episodes tally results
for i_episode in range(1000):
    obs1 = env.reset()
    epsilon = epsUpdate * i_episode
    rTally = np.array([0])
    
    for t in range(300):
        env.render()
        
        # Add action selection by max output of network over time, update epsilon slowly
        if np.random.uniform(0,1,1) >= (epsilon):
            action = env.action_space.sample()                    # this is a random action
        else:
            action = int(np.argmax(model.predict(obs1[np.newaxis,])[0]))
                
        obs2, reward, done, info = env.step(action)    # apply the action to the model and check the outputs
        
        repMem = np.column_stack((repMem, np.array([obs1,action,reward,obs2])))
            
        if repMem.shape[1] > 10:
            P = max(float(1),math.floor(repMem.shape[1]/2))
        else:
            P = int(1)

        # Get random batch
        memBatch = repMem[:,np.random.choice(repMem.shape[1], int(P), replace=False)]
        #print(memBatch[0,:])
        M = int(P)
        
        # Put Q-Learning stuff here to start...
        # Q[s,a] = Q[s,a] + a(r + y max_a'(Q[s',a'] - Q[s,a]))
        # Get targets
        tgt = np.zeros(M)

        for batchNum in range(M):
            #print(batchNum)
            
            tmp = memBatch[3,batchNum]
            tmp = tmp[np.newaxis,]
            QVal = np.max(model.predict(tmp))
            
            if done:
                tgt[batchNum] = reward
            else:
                tgt[batchNum] = reward + (gamma * QVal)
                
                              
        # End batch for
        #print(type(memBatch[np.newaxis,0]))
        # Train              
        if memBatch[0,:].shape[0] == 1:
            tmp = memBatch[0,]
            tmp = tmp[0]
            tmp = tmp[np.newaxis,]
        else:
            tmp = np.vstack(memBatch[0,:])
        
        rTally = rTally + reward
        model.train_on_batch(tmp, tgt)
        
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            print(rTally)
            break

Episode finished after 36 timesteps
[ 36.]
Episode finished after 19 timesteps
[ 19.]
Episode finished after 12 timesteps
[ 12.]
Episode finished after 11 timesteps
[ 11.]
Episode finished after 11 timesteps
[ 11.]
Episode finished after 17 timesteps
[ 17.]
Episode finished after 11 timesteps
[ 11.]
Episode finished after 17 timesteps
[ 17.]
Episode finished after 9 timesteps
[ 9.]
Episode finished after 10 timesteps
[ 10.]
Episode finished after 9 timesteps
[ 9.]
Episode finished after 9 timesteps
[ 9.]
Episode finished after 10 timesteps
[ 10.]
Episode finished after 10 timesteps
[ 10.]
Episode finished after 10 timesteps
[ 10.]
Episode finished after 8 timesteps
[ 8.]
Episode finished after 10 timesteps
[ 10.]
Episode finished after 10 timesteps
[ 10.]
Episode finished after 8 timesteps
[ 8.]
Episode finished after 11 timesteps
[ 11.]
Episode finished after 9 timesteps
[ 9.]
Episode finished after 10 timesteps
[ 10.]
Episode finished after 8 timesteps
[ 8.]
Episode finished after 8 

KeyboardInterrupt: 